# Домашнее задание 5
### Андреев Никита

In [1]:
import sys
import time
import matplotlib.pyplot as plt
import pylab
import numpy as np
import scipy
import scipy.optimize as opt
import scipy.sparse
import sklearn.datasets

Функции оракулов:

In [82]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))


class Oracle:
    def __init__(self,
                 X,
                 labels,
                 stochastic=False,
                 Q_reccurent=None,
                 batch_size=None):
        self.X = X
        self.labels = labels
        self.fc = 0
        self.gc = 0
        self.hc = 0
        self.stochastic = stochastic

        if stochastic:
            self.batch_size = batch_size
            self.X_batches = None
            self.labels_batches = None
            self.batch_index = None
            self.reload()
            self.Q = 0
            self.Q_reccurent = Q_reccurent

    def get_start(self):
#         return np.random.randn(self.X.shape[1]) / 2
        return (2 * np.random.random(self.X.shape[1]) - 1) / 2

    def reload(self):
        inds = np.arange(self.X.shape[0])
        np.random.shuffle(inds)
        self.X_batches = np.split(self.X[inds],
                                  self.X.shape[0] // self.batch_size + 1)
        self.labels_batches = np.split(self.labels[inds],
                                       self.X.shape[0] // self.batch_size + 1)
        self.batch_index = -1

    def batch(self):
        if self.batch_index >= len(self.X_batches) - 1:
            self.reload()
        self.batch_index += 1
        return self.X_batches[self.batch_index], self.labels_batches[
            self.batch_index]

    def evaluate(self, w, order=0, no_function=False):
        if self.stochastic:
            X, labels = self.batch()
        else:
            X = self.X
            labels = self.labels

        Xw = X.dot(w)
        sigmoids = sigmoid(labels * Xw)

        f = 0
        if not no_function:
            f = -1 / X.shape[0] * np.sum(np.log(sigmoids))
            self.fc += 1

        if order == 0:
            return f

        grad_coeffs = labels * (1 - sigmoids)
        X1 = X.multiply(grad_coeffs.reshape(-1, 1))
        g = -1 / X.shape[0] * np.array(X1.sum(axis=0)).reshape(X.shape[1])
        self.gc += 1

        if order == 1:
            return f, g, 0

        hess_coeffs = sigmoids * (1 - sigmoids)

        def hessian_mult(x):
            self.hc += 1
            return 1 / X.shape[0] * X.transpose().dot(X.dot(x) * hess_coeffs)

        h = hessian_mult

        if order == 2:
            return f, g, h

## В качестве данных впредь будем использовать три датасета:

a1a:

In [3]:
a1a = sklearn.datasets.load_svmlight_file('../data/a1a.txt')
X = a1a[0]
dummy = scipy.sparse.csr_matrix([[1] for i in range(X.shape[0])])
X_a1a = scipy.sparse.hstack([X, dummy])
labels_a1a = a1a[1]

breast cancer: (координаты нормированы от -1 до 1)

In [4]:
breast_cancer = sklearn.datasets.load_svmlight_file('../data/breast-cancer_scale.txt')
X = breast_cancer[0]
dummy = scipy.sparse.csr_matrix([[1] for i in range(X.shape[0])])
X_cancer = scipy.sparse.hstack([X, dummy])
labels_cancer = breast_cancer[1]-3

Случайный сгенерированный:

In [5]:
def random_dataset(alpha, beta):
    xs = np.random.normal(size=(1000, alpha.shape[0]))
    labels = np.array([np.sign(np.dot(alpha, x) + beta) for x in xs])
    return xs, labels


In [6]:
alpha = 2 * np.random.random(11) - 1
X, labels_rand = random_dataset(alpha[:-1], alpha[-1])
dummy = scipy.sparse.csr_matrix([[1] for i in range(X.shape[0])])
X_rand = scipy.sparse.hstack([X, dummy])

#### Методы из дз 2 и 3:

In [87]:
def golden_search_bounded(fun, a0, b0, eps=100 * sys.float_info.epsilon):
    ratio = (1 + 5**0.5) / 2
    a, b, c, d = a0, b0, (b0 - a0) / ratio + a0, b0 - (b0 - a0) / ratio
    fc, fd = fun(c), fun(d)
    while True:
        if b - a <= eps:
            return c
        if fc < fd:
            c, d, b = a + d - c, c, d
            fd = fc
            fc = fun(c)
        else:
            a, c, d = c, d, b - d + c
            fc = fd
            fd = fun(d)


def golden_search(fun, eps=100 * sys.float_info.epsilon):
    a, b, _, _, _, _, _ = opt.bracket(fun)
    if b < a:
        a, b = b, a
    return golden_search_bounded(fun, a, b, eps=eps)


def armijo(fun, c=0.1, k=10, f0=None, df0=None):
    x = 1

    if f0 is None:
        f0 = fun(0)

    fx = fun(x)
    fkx = fun(k * x)

    while True:
        if fx > f0 + c * df0 * x:
            x /= k
            fkx = fx
            fx = fun(x)
        elif fkx < f0 + k * c * df0 * x:
            x *= k
            fx = fkx
            fkx = fun(k * x)
        else:
            break

    return x, fx


def solveCholesky(h, d, eps=0.0001):
    G = np.array([h(x) for x in np.eye(d.shape[0])])
    if np.linalg.matrix_rank(G) < G.shape[0]:
        G = G + eps * np.eye(G.shape[0])
    L = np.linalg.cholesky(G)
    Ltx = scipy.linalg.solve_triangular(L, d, lower=True)
    return scipy.linalg.solve_triangular(np.transpose(L), Ltx, lower=False)


def solveCG(h, b, eta=0.5, policy='sqrtGradNorm'):
    if eta is None:
        eta = 0.5
    if policy is None:
        policy = 'sqrtGradNorm'

    b_norm = np.linalg.norm(b)
    if policy == 'sqrtGradNorm':
        tol = min(np.sqrt(b_norm), 0.5)
    elif policy == 'gradNorm':
        tol = min(b_norm, 0.5)
    elif policy == 'const':
        tol = eta

    eps = tol * b_norm
    x0 = np.random.random(b.shape[0]) * b_norm
    r = b - h(x0)
    p = r
    x = x0

    while True:
        rr = r.dot(r)
        hp = h(p)
        alpha = rr / np.dot(p, hp)
        x += alpha * p
        r -= alpha * hp
        if np.linalg.norm(r) < eps:
            break
        beta = r.dot(r) / rr
        p = r + beta * p

    return x


class BFGS:
    def __init__(self, H0):
        self.Hk = H0

    def update(self, s, y):
        sty = np.dot(s, y)
        Hky = self.Hk.dot(y)
        sHky = np.outer(s, Hky)
        self.Hk += (sty + np.dot(y, Hky)) / sty**2 * np.outer(
            s, s) - (sHky + sHky.T) / sty


class LBFGS:
    def __init__(self, m):
        self.ys = []
        self.ss = []
        self.stys = []
        self.m = m

    def update(self, s, y):
        self.ss.append(s)
        self.ys.append(y)
        self.stys.append(np.dot(s, y))
        if len(self.ss) > self.m:
            self.ss = self.ss[1:]
            self.ys = self.ys[1:]
            self.stys = self.stys[1:]

    def direction(self, g):
        stgs = []
        for y, s, sty in list(zip(self.ys, self.ss, self.stys))[::-1]:
            stgs.append(np.dot(s, g))
            g -= y * stgs[-1] / sty
        g *= self.stys[-1] / np.dot(self.ys[-1], self.ys[-1])
        for y, s, sty, stg in zip(self.ys, self.ss, self.stys, stgs[::-1]):
            g += (-np.dot(y, g) + stg) / sty * s
        return g

### липшиц

In [124]:
def lipschitz(fun, fk, xk, gk, L0, l):
    L = L0
    while True:

        def prox(c):
            if c > l / L:
                return c - l / L
            elif c < -l / L:
                return c + l / L
            else:
                return 0

        def diff_prox(d_i, c, i):
            if c > l / L:
                return d_i - l / L
            elif c < -l / L:
                return d_i + l / L
            else:
                return -xk[i]


        y = xk - 1 / L * gk
        diff = - 1 / L * gk
        diff = np.array([diff_prox(diff[i], y[i], i) for i in range(y.shape[0])])
        y = np.array([prox(y_i) for y_i in y])
        fx = fun(y)
        if fx < fk + np.dot(gk, diff) + L / 2 * np.dot(diff, diff):
            break
        L *= 2

    return y, fx, L

## Градиентный спуск и метод Ньютона:

Разница только в выборе направления - поэтому всё в одном методе

Параметры:

1. oracle - оракул
2. start - начальная точка
3. method - 'gradient descent', 'newton', 'BFGS' или 'L-BFGS'
4. __linear_solver - 'cg' или 'cholesky'__
5. __solver_kwargs - аргументы для linear_solver__
6. H0 - начальное значение B^-1 для BFGS (по умолчанию - E)
6. m - длина истории для L-BFGS
6. one_dim_search - способ выбора шага:
    'unit step',
    'brent',
    'golden',
    'armiho',
    'nester',
    'wolf',
    или пользовательский метод
7. args - аргументы для оракула (X, labels, outers(опционально))
8. search_kwargs - аргументы для метода одномерной оптимизации
9. условие(я) остановки: epsilon, max_iter, max_time, max_oracle


Возвращает:

1. x - результат
2. iterations - range от 0 до количества итераций
3. times - массив времён от начала вычисления до текущего шага
4. values - массив значений функции
5. grad_ratios - массив отношений $||\nabla f(w_k)||^2/||\nabla f(w_0)||^2$
6. fcalls - масиив количества вызовов функции на каждой итерации
7. gcalls - аналогично для градиента
8. hcalls - аналогично для гессиана

В реализации в main.py возвращаются только последние элементы перечисленных массивов

In [9]:
def optimization_task(oracle, method='gradient', solver_kwargs=dict([]), H0=None,
                      m=20, one_dim_search=None, search_kwargs={},
                      epsilon=0, max_iter=float('inf'), max_time=float('inf')):
    iterations, times, grad_ratios = [], [], []
    start_time, k = time.time(), 0
    x = oracle.get_start()

    if method == 'gradient':
        ord = 1
        if one_dim_search is None:
            one_dim_search = 'armijo'
    elif method == 'newton' or method == 'hfn':
        ord = 2
        if one_dim_search is None:
            one_dim_search = 'constant step'
    elif method == 'BFGS' or method == 'L-BFGS':
        ord = 1
        init = True
        if one_dim_search is None:
            one_dim_search = 'constant step'
    elif method == 'lasso':
        ord = 1
        one_dim_search = 'lipschitz'

    f0, g0, _ = oracle.evaluate(x, order=1)
    fk = f0

    if one_dim_search == 'lipschitz':
        L, L0, l = 2, 0, 0
        if 'l' in search_kwargs.keys():
            l = search_kwargs['l']
        if 'L0' in search_kwargs.keys():
            L0 = 2 * search_kwargs['L0']
            L = L0

    while True:

        _, gk, hk = oracle.evaluate(x, order=ord, no_function=True)

        ratio = np.dot(gk, gk) / np.dot(g0, g0)
        iterations.append(k)
        times.append(time.time() - start_time)
        grad_ratios.append(ratio)

        if ratio <= epsilon or k > max_iter or time.time() - start_time > max_time:
            if one_dim_search != 'armijo' and one_dim_search != 'wolfe' and one_dim_search != 'lipschitz':
                fk = oracle.evaluate(x)
            break

        k += 1

        if method == 'gradient':
            d = -gk
        elif method == 'hfn':
            d = solveCG(hk, -gk, **solver_kwargs)
        elif method == 'newton':
            d = solveCholesky(hk, -gk, **solver_kwargs)
        elif method == 'BFGS':
            if init:
                init = False
                bfgs = BFGS(H0 if H0 is not None else np.eye(x.shape[0]))
            else:
                bfgs.update(x - prev_x, gk - prev_gk)
            d = -bfgs.Hk.dot(gk)
        elif method == 'L-BFGS':
            if init:
                init = False
                lbfgs = LBFGS(m)
                d = -gk
            else:
                lbfgs.update(x - prev_x, gk - prev_gk)
                d = -lbfgs.direction(np.copy(gk))

        fun = lambda alpha: oracle.evaluate(x + d * alpha)

        if one_dim_search == 'constant step':
            alpha = 1
            if 'alpha' in search_kwargs.keys():
                alpha = search_kwargs['alpha']
        elif one_dim_search == 'golden_search':
            alpha = golden_search(fun, **search_kwargs)
        elif one_dim_search == 'brent':
            solution = opt.minimize_scalar(fun)
            alpha = solution['x']
        elif one_dim_search == 'armijo':
            alpha, fk = armijo(fun, **search_kwargs, f0=fk, df0=np.dot(gk, d))
        elif one_dim_search == 'wolfe':
            f_for_wolf = lambda z: oracle.evaluate(z)
            g_for_wolf = lambda z: oracle.evaluate(z, order=1, no_function=True)[1]
            solution = opt.line_search(f_for_wolf, g_for_wolf, x, d, **search_kwargs, gfk=gk, old_fval=fk)
            alpha = solution[0]
            if alpha is None:
                alpha = 1
            fk = solution[3]
            if fk is None:
                fk = fun(1)
        elif one_dim_search == 'lipschitz':
            x, fk, L = lipschitz(lambda z: oracle.evaluate(z), fk, x, gk, L0=max(L / 2, L0), l=l)
            continue
        else:
            alpha = one_dim_search(fun)

        if method == 'BFGS' or method == 'L-BFGS':
            prev_gk = gk
            prev_x = x

        x = x + d * alpha

    return dict([('x', x), ('nit', k), ('fun', fk), ('jac', gk), ('time', time.time() - start_time), ('ratio', ratio),
                 ('nfev', oracle.fc), ('njev', oracle.gc), ('nhev', oracle.hc), ('i', iterations), ('t', times),
                 ('r', grad_ratios)])


#### Метод для построения нескольких графиков (десятичных) логарифмов функций

In [34]:
def graph_several(xs, ys, labels, end=None, beg=0, x_l=None, y_l=None, title=None):
    pylab.rcParams['figure.figsize'] = 15, 7
    pylab.subplot(1, 1, 1)
    if end is None:
        end = max([max(x) for x in xs])
    ends = [np.argmin([np.abs(p - end) for p in x]) for x in xs]
    begs = [np.argmin([np.abs(p - beg) for p in x]) for x in xs]
    xs1 = [xs[i][begs[i]:ends[i]+1] for i in range(len(xs))]
    ys1 = [ys[i][begs[i]:ends[i]+1] for i in range(len(xs))]

    def logs(y):
        return [np.log10(v) for v in y]

    for i in range(len(xs)):
        pylab.plot(xs1[i], logs(ys1[i]), label=labels[i])

    pylab.title(title)
    pylab.xlabel(x_l)
    pylab.ylabel("$\log_{10}(r_k)$")
    pylab.grid()
    pylab.legend()
    pylab.show()

In [35]:
def graph_several2(xs,
                   ys,
                   labels,
                   end1=None,
                   beg1=0,
                   end2=None,
                   beg2=0,
                   x_l=None,
                   y_l=None,
                   title=None):
    pylab.rcParams['figure.figsize'] = 15, 7
    pylab.subplot(1, 2, 1)
    if end1 is None:
        end1 = max([max(x) for x in xs])
    ends = [np.argmin([np.abs(p - end1) for p in x]) for x in xs]
    begs = [np.argmin([np.abs(p - beg1) for p in x]) for x in xs]
    xs1 = [xs[i][begs[i]:ends[i] + 1] for i in range(len(xs))]
    ys1 = [ys[i][begs[i]:ends[i] + 1] for i in range(len(xs))]

    def logs(y):
        return [np.log10(v) for v in y]

    for i in range(len(xs)):
        pylab.plot(xs1[i], logs(ys1[i]), label=labels[i])

    pylab.title(title)
    pylab.xlabel(x_l)
    pylab.ylabel(y_l)
    pylab.grid()
    pylab.legend()

    pylab.subplot(1, 2, 2)
    if end2 is None:
        end2 = max([max(x) for x in xs])
    ends = [np.argmin([np.abs(p - end2) for p in x]) for x in xs]
    begs = [np.argmin([np.abs(p - beg2) for p in x]) for x in xs]
    xs1 = [xs[i][begs[i]:ends[i] + 1] for i in range(len(xs))]
    ys1 = [ys[i][begs[i]:ends[i] + 1] for i in range(len(xs))]

    for i in range(len(xs)):
        pylab.plot(xs1[i], logs(ys1[i]), label=labels[i])

    pylab.title(title)
    pylab.xlabel(x_l)
    pylab.ylabel("$\log_{10}(r_k)$")
    pylab.grid()
    pylab.legend()

    pylab.show()

#### Минимальные значения функций с помощью встроенных методов:

## Сравнение стратегий линейного поиска:

In [83]:
a1a = Oracle(X_a1a, labels_a1a)
cancer = Oracle(X_cancer, labels_cancer)

In [129]:
res1 = optimization_task(a1a,
                         method='lasso',
                         max_time=0.1,
                         search_kwargs={'l': 0.1}
                        )
res2 = optimization_task(a1a,
                         method='BFGS',
#                          search_kwargs={'l': 0.01},
#                          search_kwargs={'alpha': 0.1},
                         max_time=0.1)

C:\Users\Nikita\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: RuntimeWarning: invalid value encountered in double_scalars


KeyboardInterrupt: 

In [ ]:
i1, t1, r1 = res1['i'], res1['t'], res1['r']
i2, t2, r2 = res2['i'], res2['t'], res2['r']

In [ ]:
graph_several([t1, t2], [r1, r2], ['asd', 'sdf'])